# VacationPy
---

In [1]:
# Dependencies And Setup
import hvplot.pandas
import pandas as pd
import requests
from pprint import pprint # Utilized For Indexing The Geoapify API JSON Data


# Import API key
from api_keys import geoapify_key

### Load Previously Generated Weather and Coordinate Data ###

In [2]:
# Load CSV File Containing City Weather Data From WeatherPy Into Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display Sample Weather Data
city_data_df.head()

,City,Latitude,Longitude,Maximum Temperature (°C),Humidity (%),Cloudiness (%),Wind Speed (m/s),Country,Date
City_ID,,,,,,,,,
0,port-aux-francais,-49.3500,70.2167,5.70,76,59,11.85,TF,1708796841
1,jamestown,42.0970,-79.2353,-1.38,49,55,0.45,US,1708796842
2,zeya,53.7500,127.2667,-25.29,100,78,1.48,RU,1708796842
3,hawaiian paradise park,19.5933,-154.9731,20.50,85,20,2.57,US,1708796842
4,utrik,11.2278,169.8474,25.32,80,100,7.84,MH,1708796842


### Create Geographical Map for every City in `city_data_df` DataFrame ###

In [3]:
%%capture --no-display

# Configure The Map Plot
city_data_df_map = city_data_df.hvplot.points(x='Longitude',
                           y='Latitude',
                           xlabel='Longitude',
                           ylabel='Latitude',
                           geo=True,
                           tiles='OSM',
                           frame_width = 700,
                           frame_height = 500,
                           size='Humidity (%)',
                           scale=0.8,
                           alpha=0.5,
                           color='City'
                           )

# Display The Map
city_data_df_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity (%))

### Truncate `city_data_df` DataFrame based on Ideal Weather Conditions ###

In [5]:
# Narrow Down 'city_data_df' To Cities That Fit Ideal Weather Criteria
ideal_city_data_df = city_data_df.loc[(city_data_df['Maximum Temperature (°C)'] <= 26) &
                                      (city_data_df['Maximum Temperature (°C)'] >= 20) &
                                      (city_data_df['Humidity (%)'] >= 70) &
                                      (city_data_df['Wind Speed (m/s)'] <= 10.5) &
                                      (city_data_df['Cloudiness (%)'] <= 35)
                                       ]

# Drop Any Rows With Null Values
ideal_city_data_df.dropna(how='any')

# Display Ideal Weather City Data
ideal_city_data_df

,City,Latitude,Longitude,Maximum Temperature (°C),Humidity (%),Cloudiness (%),Wind Speed (m/s),Country,Date
City_ID,,,,,,,,,
3,hawaiian paradise park,19.5933,-154.9731,20.50,85,20,2.57,US,1708796842
56,cockburn town,21.4612,-71.1419,24.76,71,4,5.05,TC,1708796861
65,port elizabeth,-33.9180,25.5701,21.17,83,0,3.60,ZA,1708796864
71,coquimbo,-29.9533,-71.3436,21.85,73,0,7.20,CL,1708796596
85,mount isa,-20.7333,139.5000,25.87,73,10,4.12,AU,1708796871
110,lihue,21.9789,-159.3672,22.77,76,20,3.60,US,1708796878
164,tsiombe,-25.3000,45.4833,24.30,82,20,8.09,MG,1708796904
214,saipan,15.1355,145.7010,25.39,83,20,5.14,MP,1708796928
242,cabugao,17.7931,120.4578,23.42,82,0,1.83,PH,1708796937


### Create new DataFrame called `hotel_df` ###

In [6]:
# Create DataFrame Called hotel_df To Store City, Country, Coordinates, And Humidity
hotel_df = ideal_city_data_df.copy().reset_index(drop=True)
hotel_df = hotel_df[['City','Country','Latitude','Longitude','Humidity (%)']]

# Add Empty Column, "Hotel Name", To DataFrame To Store Hotel Found Using Geoapify API
hotel_df.insert(5, 'Hotel Name', '')


# Display DataFrame
hotel_df

,City,Country,Latitude,Longitude,Humidity (%),Hotel Name
0,hawaiian paradise park,US,19.5933,-154.9731,85,
1,cockburn town,TC,21.4612,-71.1419,71,
2,port elizabeth,ZA,-33.9180,25.5701,83,
3,coquimbo,CL,-29.9533,-71.3436,73,
4,mount isa,AU,-20.7333,139.5000,73,
5,lihue,US,21.9789,-159.3672,76,
6,tsiombe,MG,-25.3000,45.4833,82,
7,saipan,MP,15.1355,145.7010,83,
8,cabugao,PH,17.7931,120.4578,82,
9,kapa'a,US,22.0752,-159.3190,77,


### Utilize Geoapify API to find First Hotel Located Within 10,000 Metres of City Coordinates ###

In [8]:
# Set Filter And API Parameters To Search For Hotel
radius = 10000
categories = 'accommodation.hotel'
limit = 20
params = {
    'categories':categories,
    'filter':'',
    'bias':'',
    'limit':limit,
    'apiKey':geoapify_key
}

# Print Message For Initialization Of Hotel Search
print('Starting hotel search')

# Iterate Through hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get Latitude And Longitude From hotel_df DataFrame
    latitude = hotel_df.at[index, 'Latitude']
    longitude = hotel_df.at[index, 'Longitude']
    
    # Add Filter And Bias Parameters With Current City Latitude And Longitude To Params Dictionary
    params['filter'] = f"circle:{longitude},{latitude},{radius}"
    params['bias'] = f"proximity:{longitude},{latitude}"
    
    # Set Places Geoapify API Base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make API Request Using Params Dictionaty, Convert API Response To JSON Format
    name_address = requests.get(base_url, params=params).json()
    
    # Grab First Hotel From Results And Store Name In hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If No Hotel Found, Set Hotel Name As "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log Search Results
    print(f"{hotel_df.loc[index, 'City']} - Nearest Hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display Found Hotel Data
hotel_df

Starting hotel search
hawaiian paradise park - Nearest Hotel: No hotel found
cockburn town - Nearest Hotel: The Salt Raker Inn
port elizabeth - Nearest Hotel: Waterford Hotel
coquimbo - Nearest Hotel: Hotel Iberia
mount isa - Nearest Hotel: Ibis Styles
lihue - Nearest Hotel: Kauai Palms
tsiombe - Nearest Hotel: No hotel found
saipan - Nearest Hotel: Chalan Kanoa Beach Hotel
cabugao - Nearest Hotel: No hotel found
kapa'a - Nearest Hotel: Pono Kai Resort
hilo - Nearest Hotel: Dolphin Bay Hotel
wailua homesteads - Nearest Hotel: Hilton Garden Inn Kauai Wailua Bay
kalmunai - Nearest Hotel: Saji new food
conduaga - Nearest Hotel: No hotel found
east london - Nearest Hotel: No hotel found
kailua-kona - Nearest Hotel: Kona Seaside Hotel
myeik - Nearest Hotel: Green Eyes Hotel
malangas - Nearest Hotel: D'Haven Suites
libertad - Nearest Hotel: Horizon Inn Bed and Breakfast
sittwe - Nearest Hotel: Yuzana Aung Motel 1


,City,Country,Latitude,Longitude,Humidity (%),Hotel Name
0,hawaiian paradise park,US,19.5933,-154.9731,85,No hotel found
1,cockburn town,TC,21.4612,-71.1419,71,The Salt Raker Inn
2,port elizabeth,ZA,-33.9180,25.5701,83,Waterford Hotel
3,coquimbo,CL,-29.9533,-71.3436,73,Hotel Iberia
4,mount isa,AU,-20.7333,139.5000,73,Ibis Styles
5,lihue,US,21.9789,-159.3672,76,Kauai Palms
6,tsiombe,MG,-25.3000,45.4833,82,No hotel found
7,saipan,MP,15.1355,145.7010,83,Chalan Kanoa Beach Hotel
8,cabugao,PH,17.7931,120.4578,82,No hotel found
9,kapa'a,US,22.0752,-159.3190,77,Pono Kai Resort


### Add Hotel Name and Country as Additional Information in Hover for each City in Geographical Map ###

In [10]:
%%capture --no-display

# Configure Hotel Map Plot
hotel_map = hotel_df.hvplot.points(x='Longitude',
                           y='Latitude',
                           xlabel='Longitude',
                           ylabel='Latitude',
                           geo=True,
                           tiles='OSM',
                           frame_width = 700,
                           frame_height = 500,
                           size='Humidity (%)',
                           scale=0.8,
                           alpha=0.5,
                           color='City',
                           hover_cols=[
                            'Longitude',
                            'Latitude',
                            'City',
                            'Country',
                            'Humidity (%)',
                            'Hotel Name']
                           )

# Display The Map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (City,Humidity (%),Country,Hotel Name)